# Kubernetes Networking

## Introduction
>In the last notebook, you learnt how to manage pods using Workloads. Recall that workloads can control the lifecycle of pods; however, they are isolated from each other, and their operations are based solely on the arguments passed to them. 

To facilitate communication among them and with the cluster, Kubernetes offers networking resources that expose the pods. <br><br>

<p align=center><img src=images/K8_networking.webp width=600></p>

Notably, in the last lesson, we did not mention that each Pod has an IP address. We shall consider that here.

### Example

1. Start minikube if you are yet to.
2. Click on the following [link](https://aicore-files.s3.amazonaws.com/Cloud-DevOps/simple_deployment.yaml) to access a file named `simple_deployment.yaml`. 
3. Run the correct kubectl command to facilitate the deployment.
4. Run the following command: `kubectl get <name of the resource> <name of the pod>`, to view the description of the pods.
5. Run the following command: `kubectl get <name of the resource> <name of the pod> -o wide`, to view a more detailed description of the pods, which represents the output.

In [6]:
!kubectl

deployment.apps/hello created


In [18]:
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE
hello-868bcb8b84-575pt   1/1     Running   0          2m39s
hello-868bcb8b84-dp9gr   1/1     Running   0          2m38s
hello-868bcb8b84-vrgqj   1/1     Running   0          2m38s


In [19]:
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE     IP            NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-575pt   1/1     Running   0          2m47s   172.17.0.10   minikube   <none>           <none>
hello-868bcb8b84-dp9gr   1/1     Running   0          2m46s   172.17.0.8    minikube   <none>           <none>
hello-868bcb8b84-vrgqj   1/1     Running   0          2m46s   172.17.0.9    minikube   <none>           <none>


As can be observed, each pod has an IP address that is only visible when the wide (i.e. the more detailed) output is printed.

However, pod IP addresses in Kubernetes are not durable. Whenever an application is scaled up or down, or encounters an error and needs to be rebooted, the IP addresses disappear and need to be reassigned. This change in IP address occurs without warning. In response to this, Kubernetes utilises services.

## Services

One of the networking resources offered by Kubernetes is 'Services', which, in essence, exposes applications within the cluster or to the world. Thus, workloads and services manage the lifecycle of the Pods and their IP addresses, respectively.

Services serve as an abstraction layer on top of Pods, assigning a single virtual IP address to a specified group of Pods. Once these Pods are associated with that virtual IP address, any traffic addressed to it will be routed to the corresponding set of Pods. The set of Pods that are linked to a Service can be changed at any time; however, the IP address of the Service remains static.<br><br>

<p align=center><img src=images/K8_Services.webp></p>

Basically, the IP address changes when the pod terminates and restarts. Thus, it would be virtually impossible to always point to a dynamic IP address. Services do not point to the IP address, but to the pod, from which they can infer the corresponding IP address.

Consider what happens when a pod is deleted from the deployment resource.

### Example

1. Observe the IP addresses from the last deployment you created.
2. Delete one of the pods in the deployment resource.
3. Observe the IP addresses once more.

In [20]:
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE   IP            NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-575pt   1/1     Running   0          86m   172.17.0.10   minikube   <none>           <none>
hello-868bcb8b84-dp9gr   1/1     Running   0          86m   172.17.0.8    minikube   <none>           <none>
hello-868bcb8b84-vrgqj   1/1     Running   0          86m   172.17.0.9    minikube   <none>           <none>


In [21]:
!kubectl

pod "hello-868bcb8b84-575pt" deleted


In [22]:
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-dp9gr   1/1     Running   0          86m   172.17.0.8   minikube   <none>           <none>
hello-868bcb8b84-dz5bj   1/1     Running   0          19s   172.17.0.4   minikube   <none>           <none>
hello-868bcb8b84-vrgqj   1/1     Running   0          86m   172.17.0.9   minikube   <none>           <none>


As you can observe, the name and IP address of one of the pods have changed. Therefore, any attempt to point to the previous IP address will yield no result.

Services will help us point to the pods in the deployment resource, rather than to the individual IP addresses.

### Types of services
There are three main types of services in Kubernetes:
- `ClusterIP`: This is the default Service. It is a fixed IP that is located within the cluster, and it functions as a small load balancer among the resources in the cluster.
- `NodePort`: It is very similar to the `ClusterIP`, except that it creates a port in each node, hence the name. This port receives the traffic and redirects it to the corresponding Pods.
- `LoadBalancer`: It exposes the `Service` externally using a cloud provider's load balancer. It obtains the resources from the cloud cluster and redirects the network to the pods.

Click on this [link](https://kubernetes.io/docs/concepts/services-networking/service/#publishing-services-service-types) for additional details. 

In this lesson, we will run a practical demo of how these Services work.

### ClusterIP

As an example, we will utilise the `Deployment` we ran previously. We will use a `ClusterIP` that connects to the IP addresses of the pods inside the `Deployment`. Here is an example on using a `Service` that connects to the `Deployment` resource.

```
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  ports:
  - port: 8080
    targetPort: 8080
  selector:
    role: hello
```

Observe the `spec`s for this service:

- `ports` specify how to forward requests to the pods matching the `selector`. In this case,
    - forward requests to port `8080` of the pods.
    - make `Service` itself available under port `8080`.
- `selector` points to the resource that we intend to access.

For demonstration, we will run a single Pod, which will run an Ubuntu container inside the cluster. This way, we can connect to that cluster and connect to the service afterwards:

```
apiVersion: v1
kind: Pod
metadata:
  name: ubuntu
spec:
  containers:
  - name: ubuntu
    image: ubuntu
    args:
    - sleep
    - infinity
```

The example, very broadly, appears as follows:

<p align=center><img src=images/ClusterIP.png width=600></p>

#### Example

1. Create a Service and a pod resource using the configuration mentioned above. _Tip: you can use the same file to create both, separating both resources with three dashes (`---`)._
2.  You should still have the `Deployment` from before; otherwise, run it now.
3. Run the command to check all the pods and services: `kubectl get <your code here>`.
4. Run the command to describe the IP address pointed to by the `Service`: `kubectl <CMD> <resource> <name_of_the_resource>`.
5. Run the command to observe all the IP addresses of all pods.
6. Compare these addresses with the endpoints pointed to by the `Service`.
7. Delete one of the pods corresponding to the `Deployment` resource.
8. Compare the new addresses with the new endpoints pointed to by the `Service`. Determine if the service is still pointing to the correct IP addresses.

In [5]:
#Run both resources.
!kubectl

service/hello created
pod/ubuntu created


In [8]:
# Run a command to check all pods and services
!kubectl

NAME                         READY   STATUS    RESTARTS   AGE
pod/hello-868bcb8b84-dnq4w   1/1     Running   0          3m51s
pod/hello-868bcb8b84-j9p5g   1/1     Running   0          3m51s
pod/hello-868bcb8b84-rd6qj   1/1     Running   0          3m51s
pod/ubuntu                   1/1     Running   0          3m43s

NAME                 TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/hello        ClusterIP   10.103.95.80   <none>        8080/TCP   3m44s
service/kubernetes   ClusterIP   10.96.0.1      <none>        443/TCP    6h57m

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello   3/3     3            3           3m51s

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-868bcb8b84   3         3         3       3m51s


As shown, the service, hello, has been created, and it has a single clusterIP. It does not, however, specify the IP addresses to which it is pointing.

In [9]:
#Run a command to describe the IP addresses pointed to by the service:
!kubectl

Name:              hello
Namespace:         default
Labels:            <none>
Annotations:       <none>
Selector:          role=hello
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.103.95.80
IPs:               10.103.95.80
Port:              <unset>  8080/TCP
TargetPort:        8080/TCP
Endpoints:         172.17.0.5:8080,172.17.0.6:8080,172.17.0.7:8080
Session Affinity:  None
Events:            <none>


In [12]:
# Run a command to observe all the IP addresses of all pods.
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-dnq4w   1/1     Running   0          10m   172.17.0.7   minikube   <none>           <none>
hello-868bcb8b84-j9p5g   1/1     Running   0          10m   172.17.0.5   minikube   <none>           <none>
hello-868bcb8b84-rd6qj   1/1     Running   0          10m   172.17.0.6   minikube   <none>           <none>
ubuntu                   1/1     Running   0          10m   172.17.0.8   minikube   <none>           <none>


Compare the IP addresses with the endpoints pointed to by the `Service`.

In [13]:
# Delete one of the pods corresponding to the `Deployment` resource
!kubectl

pod "hello-868bcb8b84-dnq4w" deleted


In [14]:
# Compare the new addresses with the new endpoints pointed by the `Service`
!kubectl 
!kubectl 

NAME                     READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-j9p5g   1/1     Running   0          15m   172.17.0.5   minikube   <none>           <none>
hello-868bcb8b84-rd6qj   1/1     Running   0          15m   172.17.0.6   minikube   <none>           <none>
hello-868bcb8b84-z7fsc   1/1     Running   0          62s   172.17.0.9   minikube   <none>           <none>
ubuntu                   1/1     Running   0          15m   172.17.0.8   minikube   <none>           <none>
Name:              hello
Namespace:         default
Labels:            <none>
Annotations:       <none>
Selector:          role=hello
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.103.95.80
IPs:               10.103.95.80
Port:              <unset>  8080/TCP
TargetPort:        8080/TCP
Endpoints:         172.17.0.5:8080,172.17.0.6:8080,172.17.0.9:8080
Session Affinity:  None
Events:      

As can be observed, the IP addresses have changed, and the service is pointing to the newly created IP address.

Thus, as mentioned, `Service` will create the necessary routes to obtain the correct IP addresses, even when the pod has been rebooted.

Now, we consider an example that runs on the cluster. This will improve our understanding of the difference(s) between ClusterIP, NodePort, and LoadBalancer. We created a pod called `ubuntu`. If we run a command from the `ubuntu` container, we can go to any of the IP addresses, although the contact point will be `Service`.

#### Example

1. Execute bash inside the `ubuntu` pod: `kubectl exec -it ubuntu -- bash`. _Run this in the terminal, since the notebook does not allow interacting with the CLI._
2. Update and upgrade `ubuntu`: `apt upadte && apt upgrade`.
3. Install curl: `apt install curl`.
4. Curl the IP address corresponding to the service. In the example above, it would be `http://10.103.95.80:8080` or `http://hello:8080`
5. Repeat step 4 several times, and observe the `Hostname`.

The output should be similar to that shown below:
<p align=center><img src=images/curl.png width=300></p>

Observe that the same IP returns different hostnames. This is because `Service` acts as a load balancer within the cluster, and it searches for the first available pod.

Next, we consider NodePort and how it differs from ClusterIP. Prior to that, delete the resources that we have deployed thus far:

In [16]:
!kubectl 
!kubectl 

deployment.apps "hello" deleted
service "hello" deleted
pod "ubuntu" deleted


### NodePort

NodePort creates a port for each node. These ports are exposed to the world, enabling connections to the application through a browser.

When a service is created without specifying the type, the default service type, i.e. ClusterIP, will be utilised. The creation process of a NodePort is similar to that of a ClusterIP, except that the service type must be specified. Additionally, we may specify the `nodePort` number:

```
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  type: NodePort
  ports:
  - port: 8080
    targetPort: 8080
    nodePort: 30000
  selector:
    role: hello
```

#### Example

1. Create a .yaml file with the three resources: `ubuntu` pod, `hello` Deployment and `hello` NodePort.
2. Run the file using the right kubectl command.
3. Show all the created resources.
4. Obtain information about the node.

In [63]:
!kubectl 

deployment.apps/hello created
service/hello created
pod/ubuntu created


In [64]:
# Show all the services you created
!kubectl

NAME                         READY   STATUS    RESTARTS   AGE
pod/hello-868bcb8b84-4jzjf   1/1     Running   0          49s
pod/hello-868bcb8b84-lpvcr   1/1     Running   0          49s
pod/hello-868bcb8b84-shmqz   1/1     Running   0          49s
pod/ubuntu                   1/1     Running   0          49s

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
service/hello        NodePort    10.96.225.205   <none>        8080:30000/TCP   49s
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP          10h

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello   3/3     3            3           49s

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-868bcb8b84   3         3         3       49s


In [31]:
# Obtain information about the node
!kubectl

NAME                         READY   STATUS    RESTARTS   AGE
pod/hello-868bcb8b84-jfjx5   1/1     Running   0          23m
pod/hello-868bcb8b84-mtxt7   1/1     Running   0          23m
pod/hello-868bcb8b84-svqw9   1/1     Running   0          23m
pod/ubuntu                   1/1     Running   0          23m

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
service/hello        NodePort    10.98.208.104   <none>        8080:30000/TCP   23m
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP          9h

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello   3/3     3            3           23m

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-868bcb8b84   3         3         3       23m


As you can observe, there is no external IP address. This is because minikube is being employed as a testing environment and not a deployment environment. In a production environment, the computer in which an application is deployed would show an external IP for facilitating connections to it, which would be given by Kubernetes' NodePort. 

However, there is a workaround for obtaining the IP address in minikube:

In [38]:
!minikube service hello

|-----------|-------|-------------|---------------------------|
| NAMESPACE | NAME  | TARGET PORT |            URL            |
|-----------|-------|-------------|---------------------------|
| default   | hello |        8080 | http://192.168.49.2:30000 |
|-----------|-------|-------------|---------------------------|
🏃  Starting tunnel for service hello.
|-----------|-------|-------------|------------------------|
| NAMESPACE | NAME  | TARGET PORT |          URL           |
|-----------|-------|-------------|------------------------|
| default   | hello |             | http://127.0.0.1:53984 |
|-----------|-------|-------------|------------------------|
🎉  Opening service default/hello in default browser...
❗  Because you are using a Docker driver on darwin, the terminal needs to be open to run it.
^C
✋  Stopping tunnel for service hello.

❌  Exiting due to SVC_TUNNEL_STOP: stopping ssh tunnel: os: process already finished

╭────────────────────────────────────────────────────────────

While this is running, minikube will expose the mentioned URL to enable you connect to the service. These are essentially the steps taken by the production environment: it will export an external IP address to which you will add a port; thereafter, it will connect you to the pods in the cluster.

Before proceeding to the next `Service`, delete all the created resources.

In [62]:
!kubectl delete -f node_port.yaml

deployment.apps "hello" deleted
service "hello" deleted
pod "ubuntu" deleted


### LoadBalancer

`LoadBalancer`s offer a way to (more or less) uniformly distribute requests across `Node`s. They are often provided by cloud providers and can be specified, alternatively, via [`.spec.loadBalancerClass`](https://kubernetes.io/docs/concepts/services-networking/service/#load-balancer-class).

Although they share some similarities with NodePorts, LoadBalancers handle the task of selecting the node(s) to connect to. They create a __general__ IP address that will connect to __any__ of the available nodes. Conversely, a NodePort creates a different IP address for each node, following which you will have to connect to an individual node.

Additionally, LoadBalancers set the exposed port for you. To create one, simply use `LoadBalancer` in `spec.type`.

```
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  type: LoadBalancer
  ports:
  - port: 8080
    targetPort: 8080
  selector:
    role: hello

In [40]:
!kubectl apply -f node_port.yaml

deployment.apps/hello created
service/hello created
pod/ubuntu created


In [58]:
!kubectl get svc

NAME         TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
hello        LoadBalancer   10.99.190.249   <pending>     8080:31265/TCP   19m
kubernetes   ClusterIP      10.96.0.1       <none>        443/TCP          10h


For cloud environments, one can specify `loadBalancer`s via `metadata.annotations` (these will be discussed in more detail later) within the `Service` configuration.

A large list can be found [here](https://kubernetes.io/docs/concepts/services-networking/service/#internal-load-balancer).

## Ingress

> A `Kubernetes` object that manages external access to the services in a cluster.

![](./images/ingress-one-service.png)

### Features
- It exposes `HTTP`/`HTTPS` from outside the cluster to the `Service`s within the cluster.
- It may provide `load balancing` services.
- It may facilitate `SSL Termination` (decryption of the data sent to the cluster via `SSL`).

Note that an `ingress controller` of some sort must be installed. The available options are provided [here](https://kubernetes.io/docs/concepts/services-networking/ingress-controllers/).

To install it in `minikube`, run the following command:

In [56]:
!minikube addons enable ingress

💡  After the addon is enabled, please run "minikube tunnel" and your ingress resources would be available at "127.0.0.1"
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/controller:v1.0.0-beta.3
🔎  Verifying ingress addon...
🌟  The 'ingress' addon is enabled


To verify that the ingress controller is working properly, run

In [57]:
!kubectl get pods -n ingress-nginx

NAME                                        READY   STATUS      RESTARTS   AGE
ingress-nginx-admission-create--1-z7l8g     0/1     Completed   0          86s
ingress-nginx-admission-patch--1-hwz4q      0/1     Completed   1          86s
ingress-nginx-controller-69bdbc4d57-p5w9g   1/1     Running     0          86s


Ingress will create an nginx controller, as well as a load balancer or a NodePort, depending on the platform you use. Consider the following code, and observe the output:

In [69]:
!kubectl -n ingress-nginx get svc

NAME                                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                      AGE
ingress-nginx-controller             NodePort    10.106.163.35    <none>        80:31320/TCP,443:31400/TCP   17m
ingress-nginx-controller-admission   ClusterIP   10.104.117.146   <none>        443/TCP                      18m


### Defining resources

As has become conventional, use the `.yaml` config:

In [ ]:
---
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: minimal-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /
spec:
  rules:
  - http:
      paths:
      - path: /testpath
        pathType: Prefix
        backend:
          service:
            name: test
            port:
              number: 80

More information on Ingress will be provided in the practical sessions.

## Conclusion
At this point, you should have a good understanding of

- Kubernetes networking.
- the three main types of services in Kubernetes.
- Ingress and its features.